In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
titanic = pd.read_csv('/kaggle/input/titanic/train.csv')

df_train = pd.DataFrame(titanic)


pd.set_option('display.max_rows', 4)  # Show 4 rows
pd.set_option('display.max_columns', None)  # Show all columns


df_train


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
...,...,...,...,...,...,...,...,...,...,...,...,...
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.7500,NaN,Q


In [3]:
df_train.nunique()

PassengerId    891
Survived         2
              ... 
Cabin          147
Embarked         3
Length: 12, dtype: int64

In [7]:
print(df_train['Age'].value_counts())

24.0    30
22.0    27
        ..
23.5     1
74.0     1
Name: Age, Length: 88, dtype: int64


# Dataset 1

Dropping     Cabin, Ticket, Name.

In [4]:
pd.set_option('display.max_rows', 4)  # Show all rows

df_train1 = df_train

df_train1.drop(columns = ['Cabin', 'Ticket', 'Name'], axis=1, inplace=True)

df_train1


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
...,...,...,...,...,...,...,...,...,...
889,890,1,1,male,26.0,0,0,30.0000,C
890,891,0,3,male,32.0,0,0,7.7500,Q


In [6]:
df_train1['Embarked'].fillna('S', inplace=True)

In [7]:
df_train1['Sex'] = df_train1['Sex'].map({'male': 1, 'female': 2})    # male=1 female=2

df_train1

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.0,1,0,7.2500,S
1,2,1,1,2,38.0,1,0,71.2833,C
...,...,...,...,...,...,...,...,...,...
889,890,1,1,1,26.0,0,0,30.0000,C
890,891,0,3,1,32.0,0,0,7.7500,Q


In [8]:
df_train1['Embarked'] = df_train1['Embarked'].map({'S': 1, 'C': 2, 'Q': 3})

df_train1   # Two NaN in Embarked column... index 61 829 both survived

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.0,1,0,7.2500,1
1,2,1,1,2,38.0,1,0,71.2833,2
...,...,...,...,...,...,...,...,...,...
889,890,1,1,1,26.0,0,0,30.0000,2
890,891,0,3,1,32.0,0,0,7.7500,3


In [33]:
#Handling missing age

df_train1_age_Na = df_train1[df_train1['Age'].isna()]

df_train1 = df_train1.dropna(subset=['Age'])

In [10]:
df_train1_age_Na

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
5,6,0,3,1,NaN,0,0,8.4583,3
17,18,1,2,1,NaN,0,0,13.0000,1
...,...,...,...,...,...,...,...,...,...
878,879,0,3,1,NaN,0,0,7.8958,1
888,889,0,3,2,NaN,1,2,23.4500,1


In [11]:
df_train1_age = df_train1

df_train1['Family'] = df_train1['SibSp'] + df_train1['Parch']

/tmp/ipykernel_33/2467646141.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train1['Family'] = df_train1['SibSp'] + df_train1['Parch']


In [12]:
#Model to predict age.

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import GridSearchCV

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [13]:
X_age = df_train1_age.drop(columns = ['Age', 'PassengerId'])
y_age = df_train1_age['Age']

scaler = StandardScaler()
#scaler = MinMaxScaler()


X_age_scaled = scaler.fit_transform(X_age)
y_age_scaled = scaler.fit_transform(y_age.values.reshape(-1, 1)).ravel()


X_train, X_test, y_train, y_test = train_test_split(X_age_scaled, y_age_scaled, test_size = 0.2,random_state = 28)

In [14]:
model = RandomForestRegressor(n_estimators=50, random_state=28)
model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=50, random_state=28)

In [ ]:



y_pred = model.predict(X_test)


'''
# Reshape y_pred_scaled and y_test to 2D arrays
y_pred = y_pred.reshape(-1, 1)
y_test_scaled = y_test.reshape(-1, 1)

# Inverse transform the predicted 'Age' values and the true 'Age' values to the original scale
y_pred = scaler.inverse_transform(y_pred).ravel()
y_true = scaler.inverse_transform(y_test_scaled).ravel()
'''


y_pred = y_pred.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)


y_pred = scaler.inverse_transform(y_pred)
y_test = scaler.inverse_transform(y_test)


mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("R-squared:", r2)

In [15]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create a RandomForestRegressor instance
rf = RandomForestRegressor(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the grid search to your data
grid_search.fit(X_train, y_train)

# Get the best parameters and best estimator
best_params = grid_search.best_params_
best_rf = grid_search.best_estimator_

In [16]:
X_age = df_train1_age.drop(columns = ['Age', 'PassengerId'])
y_age = df_train1_age['Age']

scaler_X = StandardScaler()
scaler_y = StandardScaler()
#scaler = MinMaxScaler()


X_age_scaled = scaler_X.fit_transform(X_age)
y_age_scaled = scaler_y.fit_transform(y_age.values.reshape(-1, 1)).ravel()

X_train, X_test, y_train, y_test = train_test_split(X_age_scaled, y_age_scaled, test_size = 0.2,random_state = 28)

model = RandomForestRegressor(n_estimators=50, random_state=28)
model.fit(X_train, y_train)




RandomForestRegressor(n_estimators=50, random_state=28)

In [20]:
y_pred.shape

NameError: name 'y_pred' is not defined

In [17]:
y_pred = best_rf.predict(X_test)

y_pred = y_pred.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

y_pred = scaler_y.inverse_transform(y_pred)
y_test = scaler_y.inverse_transform(y_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Absolute Error: 8.653625763428659
Mean Squared Error: 124.73690911721732
R-squared: 0.4314342736762373


In [18]:
y_pred_round = np.round(y_pred)

y_pred_round

array([[28.],
       [27.],
       [10.],
       [25.],
       [44.],
       [29.],
       [46.],
       [41.],
       [29.],
       [45.],
       [31.],
       [27.],
       [29.],
       [28.],
       [37.],
       [29.],
       [17.],
       [25.],
       [29.],
       [30.],
       [32.],
       [34.],
       [42.],
       [48.],
       [35.],
       [10.],
       [32.],
       [33.],
       [47.],
       [30.],
       [19.],
       [33.],
       [14.],
       [35.],
       [39.],
       [37.],
       [35.],
       [32.],
       [37.],
       [29.],
       [32.],
       [31.],
       [10.],
       [16.],
       [ 6.],
       [31.],
       [30.],
       [ 7.],
       [34.],
       [32.],
       [ 9.],
       [24.],
       [33.],
       [32.],
       [10.],
       [32.],
       [34.],
       [25.],
       [28.],
       [ 6.],
       [30.],
       [30.],
       [30.],
       [26.],
       [46.],
       [25.],
       [33.],
       [33.],
       [55.],
       [ 6.],
       [31.],
      

In [19]:
# now for the missing data.
pd.set_option('display.max_rows', 6) 
df_train1_age_Na


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
5,6,0,3,1,NaN,0,0,8.4583,3
17,18,1,2,1,NaN,0,0,13.0000,1
19,20,1,3,2,NaN,0,0,7.2250,2
...,...,...,...,...,...,...,...,...,...
868,869,0,3,1,NaN,0,0,9.5000,1
878,879,0,3,1,NaN,0,0,7.8958,1
888,889,0,3,2,NaN,1,2,23.4500,1


In [36]:
df_train1_age_Na['Family'] = df_train1_age_Na['SibSp'] + df_train1_age_Na['Parch']

In [38]:
df_train1_age_Na

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family


In [39]:
X_age = df_train1_age_Na.drop(columns = ['Age', 'PassengerId'])


X_age_scaled = scaler.fit_transform(X_age)

y_pred_na = best_rf.predict(X_age_scaled)


ValueError: Found array with 0 sample(s) (shape=(0, 8)) while a minimum of 1 is required by StandardScaler.

In [ ]:
y_pred_na = y_pred_na.reshape(-1, 1)
y_pred_na = scaler_y.inverse_transform(y_pred_na)

In [27]:
y_pred_na

array([[25.22731927],
       [35.30984036],
       [32.10850063],
       [24.96795955],
       [20.7289845 ],
       [24.39780341],
       [30.64186247],
       [21.81332974],
       [31.22011626],
       [24.39780341],
       [23.20776407],
       [25.05383879],
       [21.81332974],
       [29.82673338],
       [32.105229  ],
       [44.13674384],
       [ 7.96004237],
       [24.39780341],
       [23.20776407],
       [21.81332974],
       [23.20776407],
       [23.20776407],
       [24.39780341],
       [27.47127787],
       [21.74577036],
       [23.20776407],
       [26.10565123],
       [20.09434907],
       [33.63097137],
       [23.83373733],
       [24.74377949],
       [17.00380954],
       [25.03113101],
       [44.13674384],
       [19.46077632],
       [16.973002  ],
       [35.10595966],
       [39.35319593],
       [21.90285099],
       [26.10565123],
       [21.81332974],
       [17.00380954],
       [25.40863966],
       [24.39780341],
       [18.76527641],
       [24

In [31]:
df_train1_age_Na.loc[:, 'Age'] = y_pred_na


/tmp/ipykernel_33/2722028552.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train1_age_Na.loc[:, 'Age'] = y_pred_na


In [32]:
df_train1_age_Na

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family,Name
5,6,0,3,1,25.227319,0,0,8.4583,3,0,25.227319
17,18,1,2,1,35.309840,0,0,13.0000,1,0,35.309840
19,20,1,3,2,32.108501,0,0,7.2250,2,0,32.108501
...,...,...,...,...,...,...,...,...,...,...,...
868,869,0,3,1,30.785257,0,0,9.5000,1,0,30.785257
878,879,0,3,1,24.397803,0,0,7.8958,1,0,24.397803
888,889,0,3,2,33.283151,1,2,23.4500,1,3,33.283151


In [38]:
y_pred_na = np.round(y_pred_na)

In [39]:
y_pred_na

array([[25.],
       [35.],
       [32.],
       [25.],
       [21.],
       [24.],
       [31.],
       [22.],
       [31.],
       [24.],
       [23.],
       [25.],
       [22.],
       [30.],
       [32.],
       [44.],
       [ 8.],
       [24.],
       [23.],
       [22.],
       [23.],
       [23.],
       [24.],
       [27.],
       [22.],
       [23.],
       [26.],
       [20.],
       [34.],
       [24.],
       [25.],
       [17.],
       [25.],
       [44.],
       [19.],
       [17.],
       [35.],
       [39.],
       [22.],
       [26.],
       [22.],
       [17.],
       [25.],
       [24.],
       [19.],
       [25.],
       [29.],
       [22.],
       [24.],
       [35.],
       [26.],
       [25.],
       [43.],
       [22.],
       [34.],
       [44.],
       [44.],
       [33.],
       [22.],
       [19.],
       [30.],
       [23.],
       [37.],
       [17.],
       [22.],
       [30.],
       [24.],
       [29.],
       [40.],
       [25.],
       [21.],
      

In [ ]:
#Grid Search:
#Grid search exhaustively searches through a specified set of hyperparameter combinations. Here's how you can use grid search:

#Randomized Search:
#Randomized search explores a random subset of the hyperparameter space. This can be more efficient when the hyperparameter space is large.